In [82]:
import numpy as np
import subprocess
import tqdm
import pandas as pd

import os
import os.path as osp

import glob

import h5py
import uproot

import torch
from torch import nn
from torch_geometric.data import Data
from torch_geometric.data import Dataset
from torch_geometric.data import DataLoader

import awkward as ak
import random

In [83]:

v_path = '/vols/cms/mm1221/Data/2e/val/raw/val.root'

In [84]:
import uproot
import matplotlib.pyplot as plt

# Open files
v_file = uproot.open(v_path)

# Inspect contents
print("Electron file contents:")
print(v_file.keys())


Electron file contents:
['tracksters;1', 'candidates;1', 'trackstersMerged;1', 'associations;1', 'simtrackstersSC;1', 'simtrackstersCP;1', 'tracks;1', 'simTICLCandidate;1', 'clusters;1']


In [85]:
keys_to_inspect = ["tracksters;1", "clusters;1", "simtrackstersCP;1", "trackstersMerged;1", "associations;1"]  # Add the keys you are interested in

for key in v_file.keys():
    if key in keys_to_inspect:
        print(f"Inspecting key: {key}")
        obj = v_file[key]
        
        # Check if the object is a tree or nested directory
        if hasattr(obj, "keys"):  # If it has keys, it can be expanded further
            print("Subkeys (branches or nested objects):")
            print(obj.keys())
        else:
            print(f"{key} is not a tree or expandable object.")
        print("-" * 50)

Inspecting key: tracksters;1
Subkeys (branches or nested objects):
['event', 'NClusters', 'NTracksters', 'time', 'timeError', 'regressed_energy', 'raw_energy', 'raw_em_energy', 'raw_pt', 'raw_em_pt', 'barycenter_x', 'barycenter_y', 'barycenter_z', 'barycenter_eta', 'barycenter_phi', 'EV1', 'EV2', 'EV3', 'eVector0_x', 'eVector0_y', 'eVector0_z', 'sigmaPCA1', 'sigmaPCA2', 'sigmaPCA3', 'id_probabilities', 'vertices_indexes', 'vertices_x', 'vertices_y', 'vertices_z', 'vertices_time', 'vertices_timeErr', 'vertices_energy', 'vertices_correctedEnergy', 'vertices_correctedEnergyUncertainty', 'vertices_multiplicity']
--------------------------------------------------
Inspecting key: trackstersMerged;1
Subkeys (branches or nested objects):
['event', 'time', 'timeError', 'regressed_energy', 'raw_energy', 'raw_em_energy', 'raw_pt', 'raw_em_pt', 'NTrackstersMerged', 'barycenter_x', 'barycenter_y', 'barycenter_z', 'barycenter_eta', 'barycenter_phi', 'EV1', 'EV2', 'EV3', 'eVector0_x', 'eVector0_y', '

In [124]:
ass = v_file['associations;1']

In [125]:
linking = ass['tsCLUE3D_recoToSim_CP'].array()

In [126]:
print(linking[13])


[[1, 0], [0, 1], [0, 1], [1, 0], [0, 1]]


In [130]:
print(t_x[590])

[81.5, 73.7, 69.8, 75.5, 69.3, 73.5]


In [90]:
print(m_x[13])

[-55.5]


In [144]:
class CCV1(Dataset):
    r'''
    Loads trackster-level features and associations for positive/negative edge creation.
    '''

    url = '/dummy/'

    def __init__(self, root, transform=None, max_events=1e8, inp='train'):
        super(CCV1, self).__init__(root, transform)
        self.step_size = 500
        self.inp = inp
        self.max_events = max_events
        self.fill_data(max_events)

    def fill_data(self, max_events):
        counter = 0
        print("### Loading tracksters data")

        # Choose paths depending on input mode
        if self.inp == 'train':
            tracksters_path = 'tracksters;1'
            associations_path = 'associations;1'
        else:
            tracksters_path = 'tracksters;1'
            associations_path = 'associations;1'

        for path in tqdm.tqdm(self.raw_paths):
            # Load tracksters features in chunks
            for array in uproot.iterate(
                f"{path}:{tracksters_path}",
                [
                    "time", "raw_energy",
                    "barycenter_x", "barycenter_y", "barycenter_z", 
                    "barycenter_eta", "barycenter_phi",
                    "EV1", "EV2", "EV3",
                    "eVector0_x", "eVector0_y", "eVector0_z",
                    "sigmaPCA1", "sigmaPCA2", "sigmaPCA3"
                ],
                step_size=self.step_size
            ):

                tmp_time = array["time"]
                tmp_raw_energy = array["raw_energy"]
                tmp_bx = array["barycenter_x"]
                tmp_by = array["barycenter_y"]
                tmp_bz = array["barycenter_z"]
                tmp_beta = array["barycenter_eta"]
                tmp_bphi = array["barycenter_phi"]
                tmp_EV1 = array["EV1"]
                tmp_EV2 = array["EV2"]
                tmp_EV3 = array["EV3"]
                tmp_eV0x = array["eVector0_x"]
                tmp_eV0y = array["eVector0_y"]
                tmp_eV0z = array["eVector0_z"]
                tmp_sigma1 = array["sigmaPCA1"]
                tmp_sigma2 = array["sigmaPCA2"]
                tmp_sigma3 = array["sigmaPCA3"]

                # Now load the associations for the same events/chunk
                # 'tsCLUE3D_recoToSim_CP' gives association arrays like [[1,0],[0,1],...]
                # Make sure we read from the same events
                assoc_array = []
                for assoc_chunk in uproot.iterate(
                    f"{path}:{associations_path}",
                    ["tsCLUE3D_recoToSim_CP"],
                    step_size=self.step_size,
                    entry_start=counter,
                    entry_stop=counter+len(tmp_time)
                ):
                    assoc_array = assoc_chunk["tsCLUE3D_recoToSim_CP"]
                    break  # Since we have a matching chunk, no need to continue
                
                # Concatenate or initialize storage
                if counter == 0:
                    self.time = tmp_time
                    self.raw_energy = tmp_raw_energy
                    self.bx = tmp_bx
                    self.by = tmp_by
                    self.bz = tmp_bz
                    self.beta = tmp_beta
                    self.bphi = tmp_bphi
                    self.EV1 = tmp_EV1
                    self.EV2 = tmp_EV2
                    self.EV3 = tmp_EV3
                    self.eV0x = tmp_eV0x
                    self.eV0y = tmp_eV0y
                    self.eV0z = tmp_eV0z
                    self.sigma1 = tmp_sigma1
                    self.sigma2 = tmp_sigma2
                    self.sigma3 = tmp_sigma3
                    self.assoc = assoc_array
                else:
                    self.time = ak.concatenate((self.time, tmp_time))
                    self.raw_energy = ak.concatenate((self.raw_energy, tmp_raw_energy))
                    self.bx = ak.concatenate((self.bx, tmp_bx))
                    self.by = ak.concatenate((self.by, tmp_by))
                    self.bz = ak.concatenate((self.bz, tmp_bz))
                    self.beta = ak.concatenate((self.beta, tmp_beta))
                    self.bphi = ak.concatenate((self.bphi, tmp_bphi))
                    self.EV1 = ak.concatenate((self.EV1, tmp_EV1))
                    self.EV2 = ak.concatenate((self.EV2, tmp_EV2))
                    self.EV3 = ak.concatenate((self.EV3, tmp_EV3))
                    self.eV0x = ak.concatenate((self.eV0x, tmp_eV0x))
                    self.eV0y = ak.concatenate((self.eV0y, tmp_eV0y))
                    self.eV0z = ak.concatenate((self.eV0z, tmp_eV0z))
                    self.sigma1 = ak.concatenate((self.sigma1, tmp_sigma1))
                    self.sigma2 = ak.concatenate((self.sigma2, tmp_sigma2))
                    self.sigma3 = ak.concatenate((self.sigma3, tmp_sigma3))
                    self.assoc = ak.concatenate((self.assoc, assoc_array))

                counter += len(tmp_time)
                if counter >= max_events:
                    print(f"Reached {max_events} events!")
                    break
            if counter >= max_events:
                break

    def download(self):
        raise RuntimeError(
            f'Dataset not found. Please download it from {self.url} and move all '
            f'*.root files to {self.raw_dir}')

    def len(self):
        return len(self.time)

    @property
    def raw_file_names(self):
        raw_files = sorted(glob.glob(osp.join(self.raw_dir, '*.root')))
        return raw_files

    @property
    def processed_file_names(self):
        return []

    def get(self, idx):
        # Extract per-event arrays
        event_time = self.time[idx]
        event_raw_energy = self.raw_energy[idx]
        event_bx = self.bx[idx]
        event_by = self.by[idx]
        event_bz = self.bz[idx]
        event_beta = self.beta[idx]
        event_bphi = self.bphi[idx]
        event_EV1 = self.EV1[idx]
        event_EV2 = self.EV2[idx]
        event_EV3 = self.EV3[idx]
        event_eV0x = self.eV0x[idx]
        event_eV0y = self.eV0y[idx]
        event_eV0z = self.eV0z[idx]
        event_sigma1 = self.sigma1[idx]
        event_sigma2 = self.sigma2[idx]
        event_sigma3 = self.sigma3[idx]
        event_assoc = self.assoc[idx]  # Shape: [N_tracksters, 2], either [1,0] or [0,1]

        # Convert to numpy
        event_time = np.array(event_time)
        event_raw_energy = np.array(event_raw_energy)
        event_bx = np.array(event_bx)
        event_by = np.array(event_by)
        event_bz = np.array(event_bz)
        event_beta = np.array(event_beta)
        event_bphi = np.array(event_bphi)
        event_EV1 = np.array(event_EV1)
        event_EV2 = np.array(event_EV2)
        event_EV3 = np.array(event_EV3)
        event_eV0x = np.array(event_eV0x)
        event_eV0y = np.array(event_eV0y)
        event_eV0z = np.array(event_eV0z)
        event_sigma1 = np.array(event_sigma1)
        event_sigma2 = np.array(event_sigma2)
        event_sigma3 = np.array(event_sigma3)
        event_assoc = np.array(event_assoc)  # e.g. [[1,0],[0,1],...]

        # Combine features
        flat_feats = np.column_stack((
            event_raw_energy,
            event_bx, event_by, event_bz,
            event_beta, event_bphi,
            event_EV1, event_EV2, event_EV3,
            event_eV0x, event_eV0y, event_eV0z,
            event_sigma1, event_sigma2, event_sigma3
        ))

        x = torch.from_numpy(flat_feats).float()

        # Create positive and negative edges
        # Identify tracksters from particle A and B
        # A: [1,0], B: [0,1]
        # We'll store the indices of each particle
        particleA = [i for i,assoc in enumerate(event_assoc) if assoc[0] == 1 and assoc[1] == 0]
        particleB = [i for i,assoc in enumerate(event_assoc) if assoc[0] == 0 and assoc[1] == 1]

        pos_edges = []
        neg_edges = []
        total_tracksters = len(event_time)

        # Function to get a random trackster from the same particle (allow self if only one)
        # Function to get a random trackster from the same particle (allow self if only one)
        def get_positive_target(curr_idx, same_particle_list):
            if not same_particle_list:
                # If no tracksters in the same particle (edge case), fall back to self
                return curr_idx
            if len(same_particle_list) == 1:
                # Only one trackster in this particle, must link to itself
                return curr_idx
            else:
                target = random.choice(same_particle_list)
                while target == curr_idx:
                    target = random.choice(same_particle_list)
                return target

        # Function to get a random trackster from the other particle
        def get_negative_target(curr_idx, other_particle_list):
            if not other_particle_list:
                # If no tracksters from the other particle, skip negative edge or link to self
                return curr_idx
            else:
                return random.choice(other_particle_list)

        # Create edges for each trackster
        for i in range(total_tracksters):
            # Determine which particle this trackster belongs to
            if i in particleA:
                pos_target = get_positive_target(i, particleA)
                neg_target = get_negative_target(i, particleB)
            else:
                # belongs to particle B
                pos_target = get_positive_target(i, particleB)
                neg_target = get_negative_target(i, particleA)

            pos_edges.append([i, pos_target])
            neg_edges.append([i, neg_target])

        x_pos_edge = torch.tensor(pos_edges, dtype=torch.long)
        x_neg_edge = torch.tensor(neg_edges, dtype=torch.long)
        
        return Data(x=x, x_pe=x_pos_edge, x_ne=x_neg_edge)

In [145]:
ipath = "/vols/cms/mm1221/Data/2e/val/"

In [146]:
data_train = CCV1(ipath, max_events=4000, inp = 'val')

### Loading tracksters data


  0%|                                                     | 0/1 [00:00<?, ?it/s]

Reached 4000 events!


In [147]:
print(data_train)

CCV1(4000)


In [151]:
print(data_train[591].x)

tensor([[ 5.9801e+01, -2.2472e+00, -5.7537e+01,  3.3584e+02,  2.4639e+00,
         -1.6098e+00,  3.3060e+01,  5.9419e-01,  3.1384e-01,  5.9054e-03,
         -1.5061e-01,  9.8858e-01,  5.7497e+00,  7.7084e-01,  5.6156e-01],
        [ 1.4463e+02,  1.1035e+01, -5.9803e+01,  3.3655e+02,  2.4122e+00,
         -1.3883e+00,  3.8884e+01,  2.6265e-01,  1.3752e-01,  2.5188e-02,
         -1.7028e-01,  9.8507e-01,  6.2350e+00,  5.1282e-01,  3.8272e-01]])


In [152]:
# Iterate through all events and print positive edges
for i in range(len(data_train)):
    print(f"Event {i}: Positive Edges (x_pe)")
    print(data_train[i].x_pe)  # Print the positive edges for event i
    print("-" * 50)  # Separator for readability

Event 0: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 1: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 2: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 3: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 4: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 5: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 6: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 7: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 8: Positiv

tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 2]])
--------------------------------------------------
Event 106: Positive Edges (x_pe)
tensor([[0, 3],
        [1, 3],
        [2, 2],
        [3, 1]])
--------------------------------------------------
Event 107: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 108: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 109: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 110: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 111: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 112: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
E

tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 217: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 218: Positive Edges (x_pe)
tensor([[0, 0]])
--------------------------------------------------
Event 219: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 220: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 221: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 1]])
--------------------------------------------------
Event 222: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 2]])
--------------------------------------------------
Event 223: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 224: Positi

tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 329: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 330: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 331: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 332: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 333: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 334: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 335: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 3],
        [2, 0],
        [3, 1]])
--------------------------------------------------
Event 336: Positive Edges (x_pe)


tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 440: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 441: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 442: Positive Edges (x_pe)
tensor([[0, 1],
        [1, 2],
        [2, 0]])
--------------------------------------------------
Event 443: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 444: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 445: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1],
        [2, 0]])
--------------------------------------------------
Event 446: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 447: Positive Edges (x_pe)
tensor([[0, 0],


...

tensor([[0, 3],
        [1, 1],
        [2, 0],
        [3, 4],
        [4, 0]])
--------------------------------------------------
Event 3521: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 3522: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3523: Positive Edges (x_pe)
tensor([[0, 0]])
--------------------------------------------------
Event 3524: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3525: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3526: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3527: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 3]])
---------------------------------------------

tensor([[0, 0],
        [1, 1],
        [2, 2],
        [3, 3]])
--------------------------------------------------
Event 3630: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3631: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3632: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3633: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3634: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3635: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3636: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3637: Positive Edges

tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 3741: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2]])
--------------------------------------------------
Event 3742: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3743: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2]])
--------------------------------------------------
Event 3744: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3745: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3746: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3747: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3748: Positive Edges (x_pe)
tensor([

tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3851: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2],
        [3, 3]])
--------------------------------------------------
Event 3852: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2]])
--------------------------------------------------
Event 3853: Positive Edges (x_pe)
tensor([[0, 3],
        [1, 2]])
--------------------------------------------------
Event 3854: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3855: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2]])
--------------------------------------------------
Event 3856: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2]])
--------------------------------------------------
Event 3857: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3858: Positive Edges (x_pe)
tensor([[0, 0],
        

tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 3]])
--------------------------------------------------
Event 3961: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1],
        [3, 3]])
--------------------------------------------------
Event 3962: Positive Edges (x_pe)
tensor([[0, 2],
        [1, 1]])
--------------------------------------------------
Event 3963: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1]])
--------------------------------------------------
Event 3964: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 3965: Positive Edges (x_pe)
tensor([[0, 3],
        [1, 2],
        [2, 1]])
--------------------------------------------------
Event 3966: Positive Edges (x_pe)
tensor([[0, 0],
        [1, 1],
        [2, 2]])
--------------------------------------------------
Event 3967: Positive Edges (x_pe)
tensor([[0, 3],
        [1, 3]])
-----------------------------